In [1]:
import pandas as pd
import numpy as np
import pathlib 
import os 
import main
from datetime import datetime


%precision %.2f

'%.2f'

In [2]:
################    1. tabulate model estimation result (on different algos, hyper-parameters)    ##################


# which countries to include
country_names = main.TOTAL_COUNTRY_NAMES

# results from AEJ, available for 15 countries
table_four = [[3.52, -0.83, 0.75, -3.38, 0.69, 0.0059, 0.0035, 998.06],
            [-1.15, 0.18, 1.58, -1.97, 0.34, 0.0017, 0.0013, 1290.36],
            [-0.43, 0.06, 1.41, -2.82, 0.41, 0.0006, 0.0008, 1481.87],
            [-0.50, 0.07, 1.12, -1.83, 0.29, 0.0004, 0.0005, 1567.52],
            [4.50, -1.16, 0.34, -5.54, 1.32, 0.0055, 0.0051, 978.67],
            [-0.94, 0.14, 1.43, -3.37, 0.56, 0.0011, 0.0007, 1401.06],
            [-0.50, 0.07, 1.09, -4.88, 0.81, 0.0007, 0.0006, 1461.71],
            [-0.72, 0.13, 1.10, -1.07, 0.18, 0.0008, 0.0008, 1450.38],
            [-0.65, 0.11, 1.44, -3.86, 0.65, 0.0016, 0.0011, 1304.45],
            [0.17, -0.03, 1.05, -1.45, 0.31, 0.0054, 0.0037, 1023.43],
            [-0.38, 0.05, 1.15, -4.47, 0.69, 0.0004, 0.0003, 1600.79],
            [-1.78, 0.25, 2.20, -1.86, 0.15, 0.0024, 0.0023, 1161.84],
            [-0.47, 0.06, 1.65, -2.82, 0.45, 0.0012, 0.0010, 1332.3],
            [-0.47, 0.08, 0.96, -4.39, 0.78, 0.0009, 0.0009, 1429.16],
            [-0.98, 0.18, 1.56, -1.50, 0.26, 0.0060, 0.0026, 1028.71]]

# hyper-parameters for each data version
BATCH_NUM_MAP = { 
                "AEJ_data":{1 : {'X[99]':1, 'ln(derive)':1}, 
                            2 : {'X[99]':0, 'ln(derive)':1},
                            3 : {'X[99]':1, 'ln(derive)':0},
                            4 : {'X[99]':0, 'ln(derive)':0}}, 
                "EXT_data":{1 : {'X[99]':0, 'ln(derive)':1}, 
                            2 : {'X[99]':0, 'ln(derive)':0}},
                }

# column names of tabulation table
col_names = ['batch', 'ALP', 'BET', 'SIG', 'ALPH', 'BETH', 'SIG1', 'SIG2', 'LLK', 'ALGO', 'X[99]', 'ln(derive)', 'time (days)', 'finished']

# DATA_V = 'AEJ_data'
DATA_V = 'EXT_data'


# folder to read
# DATE_FOLDER = "2021_8_11_13_17"
DATE_FOLDER = "2021_8_20"


In [3]:
# stack estimation results to a table, per country
# estimations, hyperparameters, algo, time-elapsed

for i, country in enumerate(country_names):
    # print(f"{country}")

    res = [[0] + table_four[i] + ['PS', 1, 1, 0, 1]]
    batch_num_map = BATCH_NUM_MAP[DATA_V]
    
    for batch_num in batch_num_map:
        path = f"logs/{DATA_V}/{batch_num}"
        algos = [name for name in os.listdir(path)]
        for algo in algos:
            est_path = f"{path}/{algo}/{country}"

            # default is to read latest estimation log, i.e. os.listdir(est_path)[-1]
            # file_path = f"{est_path}/{os.listdir(est_path)[-1]}"

            file_path = f"{est_path}/{DATE_FOLDER}"
            # print(f"read {file_path}")

            try:
                # set `finished` flag
                if os.path.isfile(f"{file_path}/res.pkl"):
                    finished = 1
                else:
                    finished = 0
                # time elapsed in hours
                df = pd.read_csv(f"{file_path}/log.csv", skiprows=[0], header=None).iloc[:-1]
            except:
                continue

            t0, t1 = df.iloc[0][0], df.iloc[-1][0]
            t0, t1 = datetime.strptime(t0, '%Y_%m_%d_%H_%M'), datetime.strptime(t1, '%Y_%m_%d_%H_%M')
            time = np.round((t1-t0).total_seconds()/(3600*24), 2)

            # if country == 'Poland':
            #     print(t0, t1, time)

            # estimated parameters
            ALP, BET, SIG, ALPH, BETH, SIG1, SIG2, NLLK, _ = np.array(df.iloc[-1].values[1:]).astype(float).round(3)


            res.append([batch_num, ALP, BET, SIG, ALPH, BETH, SIG1, SIG2, -1 * NLLK, algo, batch_num_map[batch_num]['X[99]'], batch_num_map[batch_num]['ln(derive)'], time, finished])

        # break # iterate algos
    path = f"outputs/MLE/{DATA_V}/{DATE_FOLDER}"
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)
    pd.DataFrame(res, columns=col_names).to_csv(f"{path}/{country}.csv", index=False)

    # break # iterate countries

In [4]:
for country in country_names:
    print(country)
    df = pd.read_csv(f"outputs/MLE/{DATA_V}/{DATE_FOLDER}/{country}.csv")
    
    # df['time (days)'] = df['time (days)'].astype(float).round(decimals=2)
    # df = df[df['ALGO'] != "SLSQP"]
    df = df[df['X[99]'] == 0]
    df = df[df['ln(derive)'] == 0]
    df.to_csv(f"outputs/MLE/{DATA_V}/{DATE_FOLDER}/{country}.csv")

    display(df)

Brazil


,batch,ALP,BET,SIG,ALPH,BETH,SIG1,SIG2,LLK,ALGO,X[99],ln(derive),time (days),finished
3,2,0.494,-0.125,0.689,-2.006,0.453,0.026,0.009,1370.326,trust-constr,0,0,1.31,1
4,2,-0.980,0.180,1.560,-1.500,0.260,0.006,0.003,-25822.696,SLSQP,0,0,0.00,1


Bulgaria


,batch,ALP,BET,SIG,ALPH,BETH,SIG1,SIG2,LLK,ALGO,X[99],ln(derive),time (days),finished
3,2,-0.979,0.181,1.560,-1.500,0.260,0.002,0.002,1223.517,trust-constr,0,0,1.50,1
4,2,-0.433,0.101,1.084,-1.672,0.302,0.002,0.001,1328.749,SLSQP,0,0,0.08,1


Chile


...

,batch,ALP,BET,SIG,ALPH,BETH,SIG1,SIG2,LLK,ALGO,X[99],ln(derive),time (days),finished
3,2,-0.273,0.083,0.991,-3.365,0.642,0.002,0.002,1625.813,trust-constr,0,0,0.93,1
4,2,-0.273,0.083,0.991,-3.364,0.642,0.002,0.002,1625.813,SLSQP,0,0,0.09,1


Thailand


,batch,ALP,BET,SIG,ALPH,BETH,SIG1,SIG2,LLK,ALGO,X[99],ln(derive),time (days),finished
3,2,-0.980,0.180,1.560,-1.500,0.26,0.006,0.003,776.530,trust-constr,0,0,2.91,0
4,2,-0.627,0.154,0.881,-3.797,0.65,0.002,0.001,952.524,SLSQP,0,0,0.03,1


Turkey


,batch,ALP,BET,SIG,ALPH,BETH,SIG1,SIG2,LLK,ALGO,X[99],ln(derive),time (days),finished
3,2,0.049,0.004,1.008,-4.137,0.908,0.004,0.002,1493.606,trust-constr,0,0,0.76,1
4,2,0.049,0.004,1.008,-4.137,0.908,0.004,0.002,1493.606,SLSQP,0,0,0.06,1


In [27]:
################   2. construct estimation table (to generate CDS risk premium)    ##############

PATH = "outputs/MLE/EXT_data/2021_8_20"

# construct dict to decide which estimation result to put in table
# dict value corresponds to the selection of i-th row in each of country table in the last cell
est_dict = {'Brazil': 0, 'Bulgaria': 1, 'Chile': 0,
            'China': 1, 'Colombia': 1, 'Croatia': 1,
            'Korea': 1, 'Malaysia': 1, 'Mexico': 1,
            'Philippines': 1, 'Poland': 1, 'Russia': 1,
            'SAfrica': 1, 'Thailand': 1, 'Turkey': 1 }


res_list = list()
for country in est_dict:
    df = pd.read_csv(f"{PATH}/{country}.csv")
    res_list.append(df.values[est_dict[country]][2:])

df = pd.DataFrame(res_list, index=country_names, columns=col_names[1:])[['ALP', 'BET', 'SIG', 'ALPH', 'BETH', 'SIG1', 'SIG2', 'LLK']]


# add start, end time, and # of obs of data used for each country
res_list = list()
for i, country in enumerate(df.index):
    _, _, _, PFILE, _, t0, t1 = main.find_overlap(ICAR_1=i, DATA_V="EXT_data")
    res_list.append([f"{t0.year}-{t0.month}-{t0.day}", f"{t1.year}-{t1.month}-{t1.day}", len(PFILE.loc[t0:t1])])

to_merge = pd.DataFrame(index=df.index, columns=['start', 'end', '# obs'], data=res_list)
df = df.merge(right=to_merge, how='left', on=df.index, left_index=True).drop(['key_0'], axis=1)

df.to_csv(f"{PATH}/estimation_table.csv")

In [28]:
df = pd.read_csv(f"{PATH}/estimation_table.csv", index_col=0)

In [29]:
df

,ALP,BET,SIG,ALPH,BETH,SIG1,SIG2,LLK,start,end,# obs
Brazil,0.494,-0.125,0.689,-2.006,0.453,0.026,0.009,1370.326,2001-11-30,2021-7-1,236
Bulgaria,-0.433,0.101,1.084,-1.672,0.302,0.002,0.001,1328.749,2003-1-31,2014-3-31,134
Chile,-0.980,0.180,1.560,-1.500,0.260,0.006,0.003,1946.725,2003-1-31,2021-7-1,222
China,-0.366,-0.004,1.793,-1.664,0.255,0.005,0.002,774.409,2003-1-31,2010-1-29,84
Colombia,0.635,-0.149,1.039,-7.000,1.359,0.003,0.002,1996.435,2003-1-31,2021-7-1,222
Croatia,0.058,-0.036,0.993,-1.887,0.386,0.003,0.002,1406.410,2002-1-31,2014-3-31,146
Korea,-0.091,-0.038,1.052,-2.891,0.449,0.003,0.002,1001.868,2002-2-28,2010-5-31,99
Malaysia,0.389,-0.139,1.050,-2.135,0.490,0.003,0.002,971.944,2002-1-31,2009-11-30,94
Mexico,0.200,-0.068,1.514,-1.894,0.284,0.005,0.003,2094.633,2001-11-30,2021-7-1,236
Philippines,0.405,-0.084,0.805,-4.655,1.057,0.006,0.004,726.144,2003-1-31,2010-5-31,88
